In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
# LOT import
from terachemcloud import TCC

#PES import
from pes import PES
from penalty_pes import Penalty_PES

#optimizer import
from eigenvector_follow import eigenvector_follow

#molecule import
from molecule import Molecule

#gsm import
from se_cross import SE_Cross

# etc
import numpy as np
from nifty import pvec1d,pmat2d,click,printcool
import manage_xyz
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

import tcc
import os   

In [2]:
##### => Job Data <= #####
tcc_options = { 
    # TCC options
    'runtype':      'gradient',
    'jobname':      'ethene test',
    'units':        'bohr',
        
    # TeraChem engine options
    'atoms':        None,
    'charge':       0,  
    'spinmult':     1,  
    'closed_shell': True,
    'restricted':   True,

    'method':       'hf',
    'basis':        '6-31gss',
        
    'precision':    'double',
    'threall':      1e-20,

    # alpha-SA-CASSCF options
    'casci':       'yes',
    'closed':       7,  
    'active':       2,  
    'cassinglets':  2,
    'fon' :  'yes',
    #'alphacas':     'yes',
    #'alpha':        0.73,
}

# Get authentication from the environment                                                                 
USER = os.environ['TCCLOUD_USER']                                                                         
API_KEY = os.environ['TCCLOUD_API_KEY']
TC = tcc.Client(url='http://fire-05-31:30080', user=USER, api_key=API_KEY, engine='terachem')

In [3]:
# level of theory
filepath='../data/ethylene.xyz'
states=[(1,0),(1,1)]
lot=TCC.from_options(states=[(1,0),(1,1)],extra_kwargs={'TC':TC,'tcc_options':tcc_options},fnm=filepath)


 initializing LOT from file


In [4]:
# => Create PES objects <= #
printcool("Building the PES objects")
pes1 = PES.from_options(lot=lot,ad_idx=states[0][1],multiplicity=states[0][0])
pes2 = PES.from_options(lot=lot,ad_idx=states[1][1],multiplicity=states[1][0])
pes = Penalty_PES(pes1,pes2,lot)

#========================================================#
#|               Building the PES objects               |#
#========================================================#
 PES1 multiplicity: 1 PES2 multiplicity: 1


In [5]:
# => Molecule <= #
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
M = Molecule.from_options(fnm=filepath,PES=pes,coordinate_type="TRIC")

#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 making primitives from options!
 making primitive Hessian
 forming Hessian in basis


In [6]:
print(M.energy)
print(M.difference_energy)

 orbfile is 
{
  "job_id": "dd469fc5-9659-425d-9714-7c04963f1308", 
  "job_status": "SUBMITTED", 
  "message": "job not finished", 
  "timestamp": null, 
  "user_id": "caldaz"
}


KeyError: 'orbfile'

In [ ]:
printcool("copy molecule \n Note that the copy from options is recommended since it properly creates new coord_obj and PES object")
newMolecule = Molecule.copy_from_options(M)

In [ ]:
# => Optimizer object <= #
ef = eigenvector_follow.from_options(DMAX=0.05,print_level=1) #Linesearch=NoLineSearch

# this is how to run optimizer by itself
#E,geoms = ef.optimize(
#    molecule=M,
#    refE=M.energy,
#    opt_type='UNCONSTRAINED',
#    opt_steps=5,
#    )

In [ ]:
# => Create GSM object <= #
printcool(" Building the GSM object")
driving_coords = [('TORSION',2,1,4,6,120.),('TORSION',2,1,4,5,180.)] 
#extra torsion here to ensure proper orientation w.r.t et_meci
gsm = SE_Cross.from_options(reactant=M,nnodes=20,driving_coords=driving_coords,DQMAG_MAX=0.5,BDIST_RATIO=0.9,optimizer=ef)

In [ ]:
# => Run GSM <= #
printcool("Starting GSM")
gsm.go_gsm(opt_steps=5)

In [ ]:
# => post processing <= #
manage_xyz.write_xyz('et_meci.xyz',gsm.nodes[gsm.nR].geometry,scale=1.)